<a href="https://colab.research.google.com/github/alexisdr/uned-tfg/blob/main/UNED-TFG-2-data-set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creación del Dataset

Durante este proceso se crea un dataset con lo datos procesados y listos para el entrenamiento del modelo.

# Parametros

*  GoldStandardTrainingEAs.txt: contiene una relación de IDS de actos clínicos seguidos de una lista de uno o varios códigos EAs, en caso de que sean aplicables, o NONE en caso de no se haya idetnificado ningún código de efecto adverso. Todos los valores están separados por espacios. Ejemplos:
```
24866017 T50.2X5A
27448436 T83.021A T83.511A T83.091A Y84.6
25205191 NONE
```
*  GoldStandardTestEAs.txt: equivalente al anterior pero con los datos del conjunto de pruebas.
*  Training: carpeta que contiene todos los informes médicos del conjunto de entrenamiento. El nombre del archivo está compuesto por el acto clínico y el identificador del informe. Ejemplo: 23062488-158483734.txt
*  Training: equivalente a training pero con el conjunto de pruebas.
* dataset_path: ruta en el que se almacenará el dataset
* labels_file: ruta en la que se almacenará la lista de etiquetas

In [53]:
ruta_base = '/drive/My Drive/CorpusPFG/'

#Datos de origen
ruta_archivo_GoldStandardTrainingEA = ruta_base + 'GoldStandardTrainingEAs.txt'
ruta_archivo_GoldStandardTestEA = ruta_base + 'GoldStandardTestEAs.txt'
ruta_archivos_entrenamiento = ruta_base + 'Training/'
ruta_archivos_test = ruta_base + 'Test/'

#Dataset procesado
ruta_dataset = ruta_base + 'Dataset'

In [95]:
!pip install -q datasets unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 KB 6.3 MB/s eta 0:00:00


Se realiza el montaje de la unidad de Google Drive para acceder a los ficheros

In [55]:
from google.colab import drive

drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


#Creación del dataset




Se crea un dataset con los tres segmentos: train, validation y test. Las etiquetas de train y validation se toman del fichero GoldStandardTrainingEAs.txt y las etiquetas del segmento test se toman del fichero GoldStandardTestEAs.txt.

Se cargan los archivos para su tratamiento.

In [56]:
import pandas as pd

column_names = ["Acto", "TieneEA_EA1", 
                "EA2", "EA3", "EA4", "EA5", "EA6", "EA7", "EA8", "EA9", "EA10"]
df_goldStandardTrainingEAs = pd.read_csv(
    ruta_archivo_GoldStandardTrainingEA, sep=' ', header=None, names=column_names)
df_goldStandardTestEAs = pd.read_csv(
    ruta_archivo_GoldStandardTestEA, sep=' ', header=None, names=column_names)

Generación de las etiquetas

In [57]:
df_goldStandardTrainingEAs

,Acto,TieneEA_EA1,EA2,EA3,EA4,EA5,EA6,EA7,EA8,EA9,EA10
0,27690409,T50.2X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,24626366,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,27967924,O90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25774939,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,24878023,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
14497,24368169,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14498,26850773,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14499,26850775,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14500,26850771,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
from datasets import ClassLabel

def crear_lista_etiquetas(dataframe):
  label_list = list(dataframe.TieneEA_EA1.unique())
  label_list.extend(list(dataframe.EA2.unique()))
  label_list.extend(list(dataframe.EA3.unique()))
  label_list.extend(list(dataframe.EA4.unique()))
  label_list.extend(list(dataframe.EA5.unique()))
  label_list.extend(list(dataframe.EA6.unique()))
  label_list.extend(list(dataframe.EA7.unique()))
  label_list.extend(list(dataframe.EA8.unique()))
  label_list.extend(list(dataframe.EA9.unique()))
  label_list.extend(list(dataframe.EA10.unique()))
  df = pd.DataFrame(label_list, columns=["label"])
  df = df.dropna()
  label_list = list(df.label.unique())
  return ClassLabel(num_classes=len(label_list), names=label_list)

class2label_entrenamiento = crear_lista_etiquetas(df_goldStandardTrainingEAs)
class2label_test = crear_lista_etiquetas(df_goldStandardTestEAs)

/drive/My Drive/CorpusPFG/Dataset/labels-train.txt
/drive/My Drive/CorpusPFG/Dataset/labels-test.txt


Precesamiento del texto de los informes

In [124]:
import os
import re
from unidecode import unidecode

JUICIO_CLINICO = 'Juicio Clínico'
TRATAMIENTO = 'Tratamiento'
FIRMADO_XXX = '_FIRMADO_XXXX_'
ETIQUETAS = ["nif", "telefono", "movil", "fecha ingreso", "fecha alta", 
             "analitica", "comentarios", "juicio","clinico","tratamiento",
             "recomendaciones","exploraciones", "complementarias"]

TRAINING = 0
TEST = 1

def omitir_textos_anonimizacion (texto):
  # busca expresiones del tipo _HOSPITAL_XXXX_ _DIRECCION_XXXX_
  return re.sub (r'_\w+_XXXX_', '', texto)

def omitir_simbolos (texto):
  # busca expresiones del tipo _HOSPITAL_XXXX_ _DIRECCION_XXXX_
  return re.sub (r'[^\w]', ' ', texto)

def es_linea_comienzo_seleccion (linea):
  return True #linea.strip() == JUICIO_CLINICO

def es_linea_fin_seleccion (linea):
  return FIRMADO_XXX in linea.strip()

def linea_tiene_contenido (linea):
  return linea.strip() != '' #and linea.strip() != JUICIO_CLINICO

def omitir_simbolos (linea):
  return linea.replace ("- ", "").replace("\n", "").replace('"', '""').strip()

def normalizar_texto (linea):
  linea = unidecode(linea) 
  linea = linea.lower()
  return linea

def quitar_etiquetas (linea):
  for etiqueta in ETIQUETAS:
    linea = linea.replace(etiqueta, "")
  return linea

def limpiar_texto (informe_medico):
  texto = ''
  seleccionar = False
  for lineaTexto in informe_medico:
    if (es_linea_comienzo_seleccion(lineaTexto)):
      seleccionar = True
    if (es_linea_fin_seleccion(lineaTexto)):
      break
    if (seleccionar):
      lineaTexto = omitir_textos_anonimizacion(lineaTexto)
      lineaTexto = omitir_simbolos(lineaTexto)
      lineaTexto = normalizar_texto(lineaTexto)
      lineaTexto = quitar_etiquetas(lineaTexto)
      if (linea_tiene_contenido(lineaTexto)):
        texto = texto + omitir_simbolos(lineaTexto) + "\n"
  return texto

def obtener_texto (acto, label_list, lista_nombres_informes_medicos, files_location, class2label):  
  texto_acto = ''
  for nombre_informe in lista_nombres_informes_medicos:
    with open (files_location + nombre_informe, 'rt') as informe_medico:
      texto_acto = texto_acto + limpiar_texto(informe_medico)
      texto_acto = texto_acto + "\n"
  return [acto,
          class2label.str2int(label_list[0].strip()), 
          label_list[0].strip(), 
          label_list, 
          lista_nombres_informes_medicos, texto_acto]

def tomar_codigos_de_todas_las_columnas(row):
  label_list = list()
  for col in column_names:
    if (col == column_names[0]):
      continue
    if (type(row[col]) == str):
      label_list.append(row[col])
  return label_list

def crear_vector_datos (dataframe, file_list, files_location, class2label): 
  dataframe_SinNone = dataframe.query("TieneEA_EA1 != 'NONE'")
  dataframe_SinNone = dataframe_SinNone.reset_index()  
  ds_resultado = []
  for index, row in dataframe_SinNone.iterrows():
    texto_filtro = str(row['Acto']) + '-'
    archivos_acto = filter(lambda x: texto_filtro in x, file_list)       
    ds_resultado.append(obtener_texto(row['Acto'], tomar_codigos_de_todas_las_columnas(row), 
                                      list(archivos_acto), files_location, class2label))
  return ds_resultado

def crear_dataframe (labeled_array):
  return pd.DataFrame(labeled_array, columns=['acto', 'label', 'label_str', 'labels_str', 'informes', 'text'])

def generate_labeled_arrays_test ():
  training_files = ["27690409-171690541.txt"]
  labeled_array_training = crear_vector_datos (
      df_goldStandardTrainingEAs.query("Acto == 27690409"), training_files, ruta_archivo_GoldStandardTrainingEA, class2label_entrenamiento)
  ds_labeled_training = crear_dataframe (labeled_array_training)
  return ds_labeled_training  

def crear_dataframe_train_test ():
  lista_archivos_entrenamiento = [f for f in os.listdir(ruta_archivo_GoldStandardTrainingEA) 
                      if os.path.isfile(os.path.join(ruta_archivo_GoldStandardTrainingEA, f))]
  vector_datos_entrenamiento = crear_vector_datos (
      df_goldStandardTrainingEAs, lista_archivos_entrenamiento, ruta_archivo_GoldStandardTrainingEA, class2label_entrenamiento)
  df_entrenamiento = crear_dataframe (vector_datos_entrenamiento)
  
  lista_datos_test = [f for f in os.listdir(ruta_archivo_GoldStandardTestEA) 
                  if os.path.isfile(os.path.join(ruta_archivo_GoldStandardTestEA, f))]
  vector_datos_test = crear_vector_datos (
      df_goldStandardTestEAs, lista_datos_test, ruta_archivo_GoldStandardTestEA, class2label_test)
  df_test = crear_dataframe (vector_datos_test)

  return df_entrenamiento, df_test



In [125]:
ds = generate_labeled_arrays_test()
ds["text"][0]

'sexo  hombre\ninforme alta de medicina interna\nvaron de 84 anos con los siguientes antecedentes personales\nno alergias medicamentosas conocidas\nexfumador hace mas de 30 anos  refiere no beber alcohol\nhta\ndiabetes mellitus tipo 2  dx en 2005  tto con glimepirida y posteriormente metformina  seguimiento irregular  ultima hba1c 6 3  en    2014\ndislipemia\nposible angina vasoespastica  dos episodios  uno en 2001  otro estudiado en h    en 2005  ambos con coronariografia normal y en los informes que aporta nunca se ha objetivado elevacion del st en el ecg durante los episodios de dolor  dolor abdominal inespecifico que no se relaciona con eventos cardiovasculares  ecocardiograma 2013  vi no dilatado  hvi moderada  fevi conservada  patron de llenado diastolico con alteracion de la relajacion  esclerosis valvular ao sin gradiente sigicativo  i ao ligera moderada  puntos de calcio en valvula mitral  i mitral ligera\ndeterioro cognitivo ligero  consulta de 2016      vertebrobasilar  2014

In [72]:
df_goldStandardTrainingEAs.query("Acto == 27690409")

,Acto,TieneEA_EA1,EA2,EA3,EA4,EA5,EA6,EA7,EA8,EA9,EA10
0,27690409,T50.2X5A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
dataframes = crear_dataframe_train_test()

Composición del dataset

In [60]:
from datasets import Dataset, Value, DatasetDict

def set_dataset_features (dataset, class2label):
  new_features = dataset.features.copy()
  new_features["label"] = class2label
  return dataset.cast(new_features)
  
dataset_entrenamiento = set_dataset_features (dataframes[TRAINING], class2label_entrenamiento)
dataset_test = set_dataset_features (dataframes[TEST], class2label_test)

dataset_entrenamiento_validacion = dataset_entrenamiento.train_test_split(test_size=0.1)

dataset_completo = DatasetDict(
    {
        "train": dataset_entrenamiento_validacion["train"],
        "validation": dataset_entrenamiento_validacion["test"],
        "test": dataset_test,
    })

dataset_completo

Casting the dataset:   0%|          | 0/2321 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/583 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['acto', 'label', 'label_str', 'labels_str', 'informes', 'text'],
        num_rows: 2088
    })
    validation: Dataset({
        features: ['acto', 'label', 'label_str', 'labels_str', 'informes', 'text'],
        num_rows: 233
    })
    test: Dataset({
        features: ['acto', 'label', 'label_str', 'labels_str', 'informes', 'text'],
        num_rows: 583
    })
})

In [61]:
dataset_completo["train"][0]

{'acto': 26557591,
 'label': 91,
 'label_str': 'T36.95XA',
 'labels_str': ['T36.95XA'],
 'informes': ['26557591-168440062.txt'],
 'text': "   \nSexo :Hombre\n\tINFORME ALTA DE MEDICINA INTERNA\n\tNIF/  \n\t  \n\t  \n\t  \n\tPaís: \n\tTeléfono:   \n\tTeléfono Móvil:   \n\tFecha ingreso:      \nFecha alta:         \n\t  \n\tTipo de Ingreso: Desde urgencias\n\tMotivo de Ingreso (MAM) hepatitis aguda colestasica toxica\nMotivo de Alta Mejoría\nMotivo de consulta dolor abdominal e ictericia\n  \n** ANTECEDENTES PERSONALES: \n· No reacciones alérgicas medicamentosas conocidas. \n· No hábitos tóxicos (tabaco, alcohol, drogas…). \n· Factores de riesgo cardiovascular Hipertensión arterial Y Dislipemia y NO Diabetes Mellitus,\n· Cardiopatía isquémica crónica: ingreso en 2004 por angina inestable (angina de mínimos esfuerzos de 1 mes de evolución, con isquemia subepicárdica anterior en ECG). DA ocluida en segmento medio, tratada con ACTP-stent recubierto con buen resultado angiográfico. ETT con F

In [62]:
dataset_completo["train"][0]['text']

"   \nSexo :Hombre\n\tINFORME ALTA DE MEDICINA INTERNA\n\tNIF/  \n\t  \n\t  \n\t  \n\tPaís: \n\tTeléfono:   \n\tTeléfono Móvil:   \n\tFecha ingreso:      \nFecha alta:         \n\t  \n\tTipo de Ingreso: Desde urgencias\n\tMotivo de Ingreso (MAM) hepatitis aguda colestasica toxica\nMotivo de Alta Mejoría\nMotivo de consulta dolor abdominal e ictericia\n  \n** ANTECEDENTES PERSONALES: \n· No reacciones alérgicas medicamentosas conocidas. \n· No hábitos tóxicos (tabaco, alcohol, drogas…). \n· Factores de riesgo cardiovascular Hipertensión arterial Y Dislipemia y NO Diabetes Mellitus,\n· Cardiopatía isquémica crónica: ingreso en 2004 por angina inestable (angina de mínimos esfuerzos de 1 mes de evolución, con isquemia subepicárdica anterior en ECG). DA ocluida en segmento medio, tratada con ACTP-stent recubierto con buen resultado angiográfico. ETT con FEVI conservada, IM leve. Rev. en Cardiología hasta 2009, ergometría en 2005, 2006 y 2009 negativas. \n· En Urología desde 2013 por elevaci

In [64]:
dataset_completo["train"].features

{'acto': Value(dtype='int64', id=None),
 'label': ClassLabel(names=['T50.2X5A', 'NONE', 'O90.0', 'T46.5X5A', 'E89.0', 'H59.022', 'Y84.2', 'T83.021A', 'K94.23', 'T50.8X5A', 'Y95', 'T50.905A', 'T50.0X5A', 'L27.0', 'T85.398A', 'T38.0X5A', 'T85.79XA', 'P39.1', 'T84.498A', 'T84.82XA', 'K91.841', 'I97.618', 'T80.1XXA', 'T84.89XA', 'T40.605A', 'R50.82', 'T81.4XXA', 'T38.3X5A', 'G97.1', 'O75.2', 'N99.820', 'T45.1X5A', 'T39.1X5A', 'T85.71XA', 'L76.02', 'K66.0', 'Y83.1', 'T82.120A', 'K12.31', 'P01.1', 'G25.1', 'G89.18', 'T38.0X5D', 'L76.31', 'M96.830', 'G97.41', 'T47.4X5A', 'N99.821', 'T45.515A', 'T80.211A', 'T46.2X5S', 'P36.9', 'T36.0X5A', 'T84.84XA', 'G62.0', 'D70.1', 'T82.868A', 'T79.6XXA', 'T40.2X5A', 'T39.395A', 'T41.5X5A', 'T84.223A', 'T45.1X5D', 'T83.498A', 'L76.22', 'T43.295A', 'K94.12', 'P39.8', 'O86.29', 'K91.71', 'T84.021A', 'M96.840', 'Y83.2', 'T87.81', 'I97.130', 'T46.0X5A', 'T84.51XA', 'P03.4', 'Y64.0', 'J95.811', 'T39.2X5A', 'G97.51', 'T48.6X5A', 'T45.615A', 'H59.021', 'T82.7XXA',

In [65]:
dataset_completo["test"].features

{'acto': Value(dtype='int64', id=None),
 'label': ClassLabel(names=['NONE', 'T38.0X5A', 'Y83.1', 'P02.69', 'T45.515A', 'T36.95XA', 'T50.2X5A', 'T39.315A', 'T39.95XA', 'O91.22', 'T86.10', 'Y95', 'T39.395A', 'T84.54XA', 'T84.226A', 'P01.1', 'K91.840', 'J95.811', 'T82.7XXA', 'T43.505A', 'T50.8X5A', 'E89.0', 'T45.1X5A', 'K94.29', 'T81.83XA', 'T45.615A', 'T86.5', 'K91.841', 'T88.8XXA', 'T80.1XXA', 'L76.22', 'O75.2', 'T46.0X5A', 'I97.190', 'O04.5', 'T81.4XXA', 'I95.2', 'T88.59XA', 'T36.1X5A', 'M96.1', 'T42.8X5A', 'O75.82', 'I97.121', 'T83.51XA', 'Y83.6', 'K66.0', 'T46.2X5A', 'Y84.2', 'L76.32', 'P03.89', 'K91.71', 'T84.428A', 'T85.49XA', 'T50.905A', 'G72.0', 'T83.89XA', 'T86.11', 'T42.4X5A', 'P03.4', 'K94.09', 'T80.212A', 'T39.2X5A', 'Y83.8', 'T39.4X5A', 'K12.31', 'P39.3', 'T82.09XD', 'K91.89', 'T82.9XXA', 'T82.868A', 'T42.75XA', 'G89.18', 'E89.89', 'R50.82', 'T50.3X5A', 'T40.2X5A', 'T82.524A', 'P13.4', 'T83.59XA', 'P02.7', 'T82.855D', 'T50.1X5A', 'P15.8', 'I97.51', 'T45.525A', 'N99.111', 'K9

In [66]:
dataset_completo.save_to_disk(ruta_dataset)

Saving the dataset (0/1 shards):   0%|          | 0/2088 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/233 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/583 [00:00<?, ? examples/s]